In [21]:
import requests
from bs4 import BeautifulSoup
import json

# BBC home page 
root_url = 'https://www.bbc.com'
response = requests.get(root_url)
doc = BeautifulSoup(response.text, 'html.parser')

def find_bbc_article(url):
    print(url)
    # BBC article page
    response_article = requests.get(url)
    soup = BeautifulSoup(response_article.text, 'html.parser')
    
    
    # Find article descrition 
    body = soup.find(property="articleBody")
    description = [p.text for p in body.find_all("p")]
    if description:
        description = '\n'.join(description)
    
    # Find articles image
    try : 
        img_url = soup.find('img',{'class':"js-image-replace"}).get('src')
    except:
        print("No Image")
    
    # Find article time
    time = soup.find(class_="date").attrs['data-seconds']
    
    return description,img_url,time

# All articles list
news_list = []

def bbc_scraper(catagory):

    

    # Find all news or articles
    newsAll = doc.find_all('div', { 'data-testid': "edinburgh-article" })
    
    # Traverse all news or articles
    for news in newsAll:
        headline = news.find('h3')
        article_path = news.find('a')

        article_url = root_url + article_path['href']

        try:
            description,img_url,time = find_bbc_article(article_url)
    
            article = {
                "id": article_path['href'],
                "source": "BBC",
                "type": catagory,
                "author": "Null",
                "title": headline.text,
                "description": description,
                "url": article_url,
                "image_url": img_url,
                "published_at": time,
                "updated_at": time
            }

            # Add the article to our list
            news_list.append(article)
        except:
            print("Url error")

catagories = ["sport","business","innovation"]#,"science_and_environment","entertainment_and_arts","health"]
politics_catagories = ["world","asia","uk"]

for catagory in catagories:
    response = requests.get(root_url +r"/"+ catagory)
    doc = BeautifulSoup(response.text, 'html.parser')
    
    # "world","asia","uk" all are politics catagory
    world = 'world'
    asia = 'asia'
    uk = 'uk'
    if catagory in politics_catagories:
        catagory='politics'

    bbc_scraper(catagory)

print("Total news:",len(news_list))

with open('bbcNews.json', 'w', encoding='utf-8') as file:
        json.dump(news_list , file, ensure_ascii=False, indent=4)




TypeError: 'NoneType' object is not subscriptable

In [6]:
print(root_url +r"/"+ catagory)

https://www.bbc.com/technology


In [13]:
print(doc)

<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>BBC Innovation | Technology, Health, Environment, AI</title><meta content="BBC Innovation | Technology, Health, Environment, AI" property="og:title"/><meta content="BBC Innovation | Technology, Health, Environment, AI" name="twitter:title"/><meta content="BBC Innovation brings you the latest in Technology news and coverage from around the world, including health, environment, AI, and more" name="description"/><meta content="BBC Innovation brings you the latest in Technology news and coverage from around the world, including health, environment, AI, and more" property="og:description"/><meta content="BBC Innovation brings you the latest in Technology news and coverage from around the world, including health, environment, AI, and more" name="twitter:description"/><meta content="#da532c" name="msapplication-TileColor"/><meta content="#ffffff" name="theme-color"/><meta content="NOO

In [12]:
newsAll = doc.find_all('div', { 'class': 'sc-4fedabc7-3 zTZri' })

In [31]:
root_url = 'https://www.bbc.com/news'
response = requests.get(root_url)
doc = BeautifulSoup(response.text, 'html.parser')

newsAll = doc.find_all('div', { 'data-testid': "edinburgh-card" })


In [32]:
print(newsAll)

[<div data-testid="edinburgh-card"><div class="sc-4befc967-0 bUlrPj" data-analytics="true"><a class="sc-4befc967-1 gzOvMy" data-testid="internal-link" href="/news/world-middle-east-68788140"><div class="sc-b8778340-0 kFuHJG" data-testid="edinburgh-article"><div class="sc-b8778340-1 jvzsLx" data-testid="card-media-wrapper"><div class="sc-b8778340-2 kUyIkJ" data-testid="card-media"><div class="sc-a898728c-1 jWZsJP"><img class="sc-a898728c-0 haCQWq hide-when-no-script" src="/bbcx/grey-placeholder.png"/><img alt="Three Israeli students speak to Jeremy Bowen" class="sc-a898728c-0 kbvxap" loading="lazy" sizes="(min-width: 768px) 33vw, 96vw" src="https://ichef.bbci.co.uk/news/480/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01fb4b58.jpg.webp" srcset="https://ichef.bbci.co.uk/news/240/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01fb4b58.jpg.webp 240w,https://ichef.bbci.co.uk/news/320/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01

In [33]:
for news in newsAll:
    article_path = news.find('a')
    print(article_path)

<a class="sc-4befc967-1 gzOvMy" data-testid="internal-link" href="/news/world-middle-east-68788140"><div class="sc-b8778340-0 kFuHJG" data-testid="edinburgh-article"><div class="sc-b8778340-1 jvzsLx" data-testid="card-media-wrapper"><div class="sc-b8778340-2 kUyIkJ" data-testid="card-media"><div class="sc-a898728c-1 jWZsJP"><img class="sc-a898728c-0 haCQWq hide-when-no-script" src="/bbcx/grey-placeholder.png"/><img alt="Three Israeli students speak to Jeremy Bowen" class="sc-a898728c-0 kbvxap" loading="lazy" sizes="(min-width: 768px) 33vw, 96vw" src="https://ichef.bbci.co.uk/news/480/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01fb4b58.jpg.webp" srcset="https://ichef.bbci.co.uk/news/240/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01fb4b58.jpg.webp 240w,https://ichef.bbci.co.uk/news/320/cpsprodpb/11258/production/_133123207_095dbce7-2bfa-4fec-94d2-c67b01fb4b58.jpg.webp 320w,https://ichef.bbci.co.uk/news/480/cpsprodpb/11258/production/_13312320

In [35]:
mytitle = newsAll[0].find('h2').text
myarticle = newsAll[0].find('a')['href']